In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:

import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, f1_score, accuracy_score, multilabel_confusion_matrix, ConfusionMatrixDisplay, confusion_matrix
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from src.data_processor.phase_3.prob1.v1 import  Phase3Prob1FeatureProcessor
import numpy as np
import gc
from collections import Counter

In [ ]:
feat_to_name = {'feature1': 'dur', 'feature2': 'proto', 'feature3': 'service', 'feature4': 'state', 'feature5': 'spkts', 'feature6': 'dpkts', 'feature7': 'sbytes', 'feature8': 'dbytes', 'feature9': 'sttl', 'feature10': 'dttl', 'feature11': 'sload', 'feature12': 'dload', 'feature13': 'sloss', 'feature14': 'dloss', 'feature15': 'sinpkt', 'feature16': 'dinpkt', 'feature17': 'sjit', 'feature18': 'djit', 'feature19': 'swin', 'feature20': 'stcpb', 'feature21': 'dtcpb', 'feature22': 'dwin', 'feature23': 'tcprtt', 'feature24': 'synack', 'feature25': 'ackdat', 'feature26': 'smean', 'feature27': 'dmean', 'feature28': 'trans_depth', 'feature29': 'response_body_len', 'feature30': 'ct_srv_src', 'feature31': 'ct_state_ttl', 'feature32': 'ct_dst_ltm', 'feature33': 'ct_src_dport_ltm', 'feature34': 'ct_dst_sport_ltm', 'feature35': 'ct_dst_src_ltm', 'feature36': 'is_ftp_login', 'feature37': 'ct_ftp_cmd', 'feature38': 'ct_flw_http_mthd', 'feature39': 'ct_src_ltm', 'feature40': 'ct_srv_dst', 'feature41': 'is_sm_ips_ports', 'label': 'label'}
name_to_feat = {v:k for k, v in feat_to_name.items()}

In [3]:
df = pd.read_parquet("F:/Data/MLOPS_2023/data_phase-3/phase-3/prob-1/raw_train.parquet")
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

(61748, 42)

In [33]:
df2 = pd.read_csv("F:/Data/MLOPS_2023/UNSW_NB15_training-set.csv")
df3 = pd.read_csv("F:/Data/MLOPS_2023/UNSW_NB15_testing-set.csv")
df2 = pd.concat([df2, df3])
del df3
gc.collect()
df2.drop(columns=['id','rate','attack_cat'],inplace=True)
df2.drop_duplicates(inplace=True)
df2.shape
df2.columns = [name_to_feat[c] for c in df2.columns]
df2 = df2.sample(frac=1, random_state=42, replace=False)


273

In [9]:
# df3 = pd.concat([df, df2])

In [22]:
HyperParameters={'n_estimators':100,
                 'learning_rate':0.1,
                 'max_depth':32,
                 'colsample_bytree':0.8,
                 'subsample':0.8,
                 'reg_alpha':1,
                 'reg_lambda':0,
                 'random_state':42}

hyper_parameters = HyperParameters.copy()

In [23]:
df['label']

0        1
1        1
2        0
3        1
4        1
        ..
61743    0
61744    1
61745    0
61746    1
61747    1
Name: label, Length: 61748, dtype: int64

In [29]:
processor = Phase3Prob1FeatureProcessor()
new_df = processor.fit_transform(df2)
kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
FEATURES = processor.data_features['features']
categorical = processor.data_features['categorical_features']
TARGET = 'label'
kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
models = []
scores = []
oofs = np.zeros(df2.shape[0])
for i, (train_idx, valid_idx) in enumerate(kfold.split(new_df, df2['label'])):
    X_train = new_df.iloc[train_idx][FEATURES]
    X_valid = new_df.iloc[valid_idx][FEATURES]
    y_train = df2.iloc[train_idx][TARGET]
    y_valid = df2.iloc[valid_idx][TARGET]
    
    # scaler = StandardScaler()
    # scaler = scaler.fit(X_train[num_col])
    # X_train[num_col] = scaler.transform(X_train[num_col])
    # X_valid[num_col] = scaler.transform(X_valid[num_col])
    # model = LogisticRegression(max_iter=7000)
    # model.fit(X_train, y_train)
    model = LGBMClassifier(**hyper_parameters)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              categorical_feature=categorical,
              eval_metric=["logloss", "auc"],
              early_stopping_rounds=50,
              verbose=50)
    models.append(model)
    y_pred_proba = model.predict_proba(X_valid)[:,1]
    oofs[valid_idx] = y_pred_proba

    print(f"fold {i} : {roc_auc_score(y_valid, y_pred_proba)}")
    scores.append(roc_auc_score(y_valid, y_pred_proba))

print(np.mean(scores), np.std(scores))

[50]	training's binary_logloss: 0.179684	training's auc: 0.979738	valid_1's binary_logloss: 0.180093	valid_1's auc: 0.97959
[100]	training's binary_logloss: 0.165232	training's auc: 0.98267	valid_1's binary_logloss: 0.169482	valid_1's auc: 0.981333
[150]	training's binary_logloss: 0.157164	training's auc: 0.984671	valid_1's binary_logloss: 0.166166	valid_1's auc: 0.981941
[200]	training's binary_logloss: 0.150513	training's auc: 0.986263	valid_1's binary_logloss: 0.163977	valid_1's auc: 0.982343
[250]	training's binary_logloss: 0.144989	training's auc: 0.987597	valid_1's binary_logloss: 0.162806	valid_1's auc: 0.982587
[300]	training's binary_logloss: 0.139953	training's auc: 0.988741	valid_1's binary_logloss: 0.16183	valid_1's auc: 0.982763
fold 0 : 0.982763140286928
[50]	training's binary_logloss: 0.178196	training's auc: 0.980009	valid_1's binary_logloss: 0.182774	valid_1's auc: 0.97879
[100]	training's binary_logloss: 0.164482	training's auc: 0.982795	valid_1's binary_logloss: 0.17

In [28]:
gmodel = LGBMClassifier(**hyper_parameters)
gmodel.fit(new_df[FEATURES], df2[TARGET],
           eval_set=[(new_df[FEATURES],df2[TARGET])],
           eval_metric=["logloss", "auc"],
           categorical_feature=categorical,
           verbose=50)

with open(f'../checkpoints/phase-3/prob-1/v1.pkl','wb') as file:
    pickle.dump(gmodel, file)

[50]	valid_0's binary_logloss: 0.179271	valid_0's auc: 0.979686
[100]	valid_0's binary_logloss: 0.164923	valid_0's auc: 0.982563
[150]	valid_0's binary_logloss: 0.157476	valid_0's auc: 0.984384
[200]	valid_0's binary_logloss: 0.151503	valid_0's auc: 0.985833
[250]	valid_0's binary_logloss: 0.146402	valid_0's auc: 0.987052
[300]	valid_0's binary_logloss: 0.142013	valid_0's auc: 0.988075
